In [1]:
!pip install geohash2

  Preparing metadata (setup.py) ... done
  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15544 sha256=d5fa0f67a3073ce4df251593035037beff9fce3449475cf168ee9635fb7ce9bf
  Stored in directory: /root/.cache/pip/wheels/c0/21/8d/fe65503f4f439aef35193e5ec10a14adc945e20ff87eb35895
Successfully built geohash2


In [2]:
!pip install pytorch_lightning > /dev/null
print("Pytorch_lightning: installed")

Pytorch_lightning: installed


In [3]:
import requests
import pandas as pd
import os
import geohash2

# Dataset preparation:

In [4]:
# @title Download and store the dataset
if False:
  # URL of the dataset text file on GitHub
  text_file_url = 'https://raw.githubusercontent.com/Paco-Danes/DeepPoi/main/dataset_NYC.txt'
  # The directory to store the file in the colab session
  directory_path = '/content/dataset/'
  file_path = os.path.join(directory_path, 'example_dataset.txt')

  # Check if directory exists, if not, create it
  if not os.path.exists(directory_path):
      os.makedirs(directory_path)

  # Download the file
  r = requests.get(text_file_url)

  # Save the content to a file
  with open(file_path, 'w') as f:
      f.write(r.text)
  print("Text file downloaded and saved to:", file_path)

else:
  file_path = "/content/DeepPoi/Data/dataset_NYC.txt"

In [5]:
!git clone https://github.com/Paco-Danes/DeepPoi.git

Cloning into 'DeepPoi'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 50 (delta 15), reused 8 (delta 2), pack-reused 14
Receiving objects: 100% (50/50), 47.43 MiB | 31.26 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [6]:
# Column names
columns = [
    'User_ID',
    'Venue_ID',
    'Venue_Category_ID',
    'Venue_Category_Name',
    'Latitude',
    'Longitude',
    'Timezone_Offset',
    'UTC_Time'
]
# Read the TSV file into a DataFrame
df = pd.read_csv(file_path, sep='\t', names = columns, encoding = "latin-1")
# Shape check of the DataFrame
df.shape

(227428, 8)

In [7]:
df.head()

,User_ID,Venue_ID,Venue_Category_ID,Venue_Category_Name,Latitude,Longitude,Timezone_Offset,UTC_Time
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,Tue Apr 03 18:00:09 +0000 2012
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,Tue Apr 03 18:00:25 +0000 2012
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,Tue Apr 03 18:02:24 +0000 2012
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,Tue Apr 03 18:02:41 +0000 2012
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,Tue Apr 03 18:03:00 +0000 2012


In [8]:
# Function to retrieve geohash at different precisions
def get_geohash(row, precision):
    return geohash2.encode(row['Latitude'], row['Longitude'], precision=precision)

# List of precisions, 2 is not present because the whole NYC is in the same cell of precision 2 (every point same G@2 cell, i checked)
precisions = [3, 4, 5, 6]

# Add new columns for each precision
for precision in precisions:
    column_name = f'g{precision}'
    df[column_name] = df.apply(lambda row: get_geohash(row, precision), axis=1)

df.head()

,User_ID,Venue_ID,Venue_Category_ID,Venue_Category_Name,Latitude,Longitude,Timezone_Offset,UTC_Time,g3,g4,g5,g6
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,Tue Apr 03 18:00:09 +0000 2012,dr5,dr5r,dr5rs,dr5rsh
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,Tue Apr 03 18:00:25 +0000 2012,dr5,dr5r,dr5r5,dr5r50
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,Tue Apr 03 18:02:24 +0000 2012,dr5,dr5r,dr5rw,dr5rws
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,Tue Apr 03 18:02:41 +0000 2012,dr5,dr5r,dr5ru,dr5ru3
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,Tue Apr 03 18:03:00 +0000 2012,dr5,dr5r,dr5ru,dr5ru2


In [9]:
print("How many different cells at different precisions does the dataset cover?\n", {x: len(df[x].unique()) for x in ['g3', 'g4', 'g5', 'g6']})

How many different cells at different precisions does the dataset cover?
 {'g3': 2, 'g4': 10, 'g5': 128, 'g6': 2483}


In [10]:
# Calculate the minimum and maximum values for latitude and longitude
min_latitude = df['Latitude'].min()
max_latitude = df['Latitude'].max()
min_longitude = df['Longitude'].min()
max_longitude = df['Longitude'].max()

# Min-max normalize latitude and longitude columns
df['Norm_Latitude'] = (df['Latitude'] - min_latitude) / (max_latitude - min_latitude)
df['Norm_Longitude'] = (df['Longitude'] - min_longitude) / (max_longitude - min_longitude)
df['Venue_ID_int'] = pd.factorize(df['Venue_ID'])[0] + 1
df['Category_ID'] = pd.factorize(df['Venue_Category_ID'])[0] + 1

for g in ['g3', 'g4', 'g5', 'g6']:
  df[g + "_ID"] = pd.factorize(df[g])[0] + 1

# Drop the original categorical column
df = df.drop(['Venue_Category_ID', 'Latitude', 'Longitude', 'Venue_ID', 'g3', 'g4', 'g5', 'g6'], axis=1)
print(df.shape)
df.head()

(227428, 12)


,User_ID,Venue_Category_Name,Timezone_Offset,UTC_Time,Norm_Latitude,Norm_Longitude,Venue_ID_int,Category_ID,g3_ID,g4_ID,g5_ID,g6_ID
0,470,Arts & Crafts Store,-240,Tue Apr 03 18:00:09 +0000 2012,0.386208,0.460596,1,1,1,1,1,1
1,979,Bridge,-240,Tue Apr 03 18:00:25 +0000 2012,0.127885,0.390219,2,2,1,1,2,2
2,69,Home (private),-240,Tue Apr 03 18:02:24 +0000 2012,0.377868,0.662835,3,3,1,1,3,3
3,395,Medical Center,-240,Tue Apr 03 18:02:41 +0000 2012,0.444161,0.494546,4,4,1,1,4,4
4,87,Food Truck,-240,Tue Apr 03 18:03:00 +0000 2012,0.432595,0.482464,5,5,1,1,4,5


In [11]:
flag = sorted(df['User_ID'].unique()) == list(range(1, df['User_ID'].max() + 1))
print('Check that user_id are all the ints between 1 and max ID: ', flag, '\n Print the first IDs: ')
# Make them start from O
if flag:
  df['User_ID'] = df['User_ID'] - 1
sorted(df['User_ID'].unique())[:11]

Check that user_id are all the ints between 1 and max ID:  True 
 Print the first IDs: 


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [12]:
df.to_csv('PreparedDataset.csv', index = False)

# Nets

In [13]:
#Testing and visualizing the preprocesing
df['UTC_Time'] = pd.to_datetime(df['UTC_Time'])
# Sort the dataframe by user_id and UTC_Time
# P.S: it looks like the dataset is already sorted by date, but it's better to be sure.
df_sorted = df.sort_values(by=['User_ID', 'UTC_Time'])
# Group by user_id and aggregate place_id into lists

In [14]:
# Group by user_id and aggregate place_id and corresponding grid cell IDs into lists
seqs = df_sorted.groupby('User_ID').apply(lambda x: {'Venue_ID_int': x['Venue_ID_int'].tolist(),
                                                                  'g3_ID': x['g3_ID'].tolist(),
                                                                  'g4_ID': x['g4_ID'].tolist(),
                                                                  'g5_ID': x['g5_ID'].tolist(),
                                                                  'g6_ID': x['g6_ID'].tolist()})
seqs

User_ID
0       {'Venue_ID_int': [2024, 2745, 3671, 4008, 4422...
1       {'Venue_ID_int': [1430, 508, 3742, 4408, 5903,...
2       {'Venue_ID_int': [626, 2564, 2702, 1998, 3377,...
3       {'Venue_ID_int': [359, 408, 408, 408, 408, 408...
4       {'Venue_ID_int': [23993, 24001, 24009, 20074, ...
                              ...                        
1078    {'Venue_ID_int': [330, 821, 849, 821, 849, 484...
1079    {'Venue_ID_int': [2329, 2634, 2876, 2329, 4159...
1080    {'Venue_ID_int': [4964, 35695, 4964, 35728, 16...
1081    {'Venue_ID_int': [615, 650, 2851, 1440, 4691, ...
1082    {'Venue_ID_int': [520, 1417, 1426, 1973, 530, ...
Length: 1083, dtype: object

In [15]:
print(type(seqs), type(seqs[0]), type(seqs[0]['g3_ID']))
seqs.shape

<class 'pandas.core.series.Series'> <class 'dict'> <class 'list'>


(1083,)

In [16]:
df.head()

,User_ID,Venue_Category_Name,Timezone_Offset,UTC_Time,Norm_Latitude,Norm_Longitude,Venue_ID_int,Category_ID,g3_ID,g4_ID,g5_ID,g6_ID
0,469,Arts & Crafts Store,-240,2012-04-03 18:00:09+00:00,0.386208,0.460596,1,1,1,1,1,1
1,978,Bridge,-240,2012-04-03 18:00:25+00:00,0.127885,0.390219,2,2,1,1,2,2
2,68,Home (private),-240,2012-04-03 18:02:24+00:00,0.377868,0.662835,3,3,1,1,3,3
3,394,Medical Center,-240,2012-04-03 18:02:41+00:00,0.444161,0.494546,4,4,1,1,4,4
4,86,Food Truck,-240,2012-04-03 18:03:00+00:00,0.432595,0.482464,5,5,1,1,4,5


In [17]:
def preprocess_dataset(df):
    # Convert UTC_Time column to datetime
    df['UTC_Time'] = pd.to_datetime(df['UTC_Time'])
    # Sort the dataframe by user_id and UTC_Time
    df_sorted = df.sort_values(by=['User_ID', 'UTC_Time'])
    # Group by user_id and aggregate place_id and grid IDs into lists
    sequences = df_sorted.groupby('User_ID').apply(lambda x: {'Venue_ID_int': x['Venue_ID_int'].tolist(),
                                                              'g3_ID': x['g3_ID'].tolist(),
                                                              'g4_ID': x['g4_ID'].tolist(),
                                                              'g5_ID': x['g5_ID'].tolist(),
                                                              'g6_ID': x['g6_ID'].tolist()})
    user_ids = sequences.index.tolist()
    # Split sequences into training and validation
    train_sequences = {}
    val_sequences = {}

    for user_id, seq_data in sequences.items():
        venue_sequence = seq_data['Venue_ID_int']
        g3_sequence = seq_data['g3_ID']
        g4_sequence = seq_data['g4_ID']
        g5_sequence = seq_data['g5_ID']
        g6_sequence = seq_data['g6_ID']

        split_index = int(0.8 * len(venue_sequence))  # 80-20 split

        train_sequences[user_id] = {'Venue_ID_int': venue_sequence[:split_index],
                                     'g3_ID': g3_sequence[:split_index],
                                     'g4_ID': g4_sequence[:split_index],
                                     'g5_ID': g5_sequence[:split_index],
                                     'g6_ID': g6_sequence[:split_index]}

        val_sequences[user_id] = {'Venue_ID_int': venue_sequence[split_index:],
                                   'g3_ID': g3_sequence[split_index:],
                                   'g4_ID': g4_sequence[split_index:],
                                   'g5_ID': g5_sequence[split_index:],
                                   'g6_ID': g6_sequence[split_index:]}

    return train_sequences, val_sequences

In [18]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, sequences_with_geohash):
        self.sequences_with_geohash = sequences_with_geohash

    def __len__(self):
        return len(self.sequences_with_geohash) # = number of users

    def __getitem__(self, idx):
        seq_dict = self.sequences_with_geohash[idx] # select a specific user
        # Construct input sequences for user_ID = idx
        inputs = ( torch.tensor(seq_dict['Venue_ID_int'][:-1]), torch.tensor(seq_dict['g3_ID'][:-1]),
                   torch.tensor(seq_dict['g4_ID'][:-1]), torch.tensor(seq_dict['g5_ID'][:-1]), torch.tensor(seq_dict['g6_ID'][:-1]) )

        targets = ( torch.tensor(seq_dict['Venue_ID_int'][1:]), torch.tensor(seq_dict['g3_ID'][1:]),
                   torch.tensor(seq_dict['g4_ID'][1:]), torch.tensor(seq_dict['g5_ID'][1:]), torch.tensor(seq_dict['g6_ID'][1:]) )

        user_id = torch.tensor(idx)

        return inputs, targets, user_id # this will be an element in the batch

In [19]:
import torch
from torch.nn.utils.rnn import pad_sequence

def custom_collate(batch):
    # Unzip the batch into separate lists of inputs, targets, and user IDs
    inputs_batch, targets_batch, user_ids_batch = zip(*batch)

    # Get the maximum length of the venue sequences in the batch
    max_venue_length = max(len(inputs[0]) for inputs in inputs_batch)  # Assuming venue sequences are always at index 0

    # Initialize lists to store padded inputs, targets, and user IDs
    padded_inputs_batch = []
    padded_targets_batch = []
    user_ids_tensor = torch.tensor(user_ids_batch)

    # Pad each sequence type in the batch to match the maximum venue length
    for inputs, targets in zip(inputs_batch, targets_batch):
        padded_inputs = []
        padded_targets = []

        padding = max_venue_length - len(inputs[0])

        for i in range(len(inputs)):
            # Pad the sequences to match the maximum venue length
            padded_input = torch.nn.functional.pad(inputs[i], (0, padding))
            padded_target = torch.nn.functional.pad(targets[i], (0, padding))
            padded_inputs.append(padded_input)
            padded_targets.append(padded_target)

        padded_inputs_batch.append(tuple(padded_inputs))
        padded_targets_batch.append(tuple(padded_targets))

    # Convert the lists of padded inputs and targets into tensors
    padded_inputs_tensor = tuple(map(torch.stack, zip(*padded_inputs_batch)))
    padded_targets_tensor = tuple(map(torch.stack, zip(*padded_targets_batch)))

    return (padded_inputs_tensor, padded_targets_tensor, user_ids_tensor)


In [20]:
import pandas as pd
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.nn.utils.rnn import pad_sequence

# Define LightningModule
class StackedLSTMClassifier(pl.LightningModule):
    def __init__(self, lengths, embedding_dim, hidden_size_lstm, num_layers_lstm, dropout_prob=0.33):
        super(StackedLSTMClassifier, self).__init__()

        self.embeddingUser = nn.Embedding(lengths['num_of_users'], embedding_dim)
        self.embeddingVenue = nn.Embedding(lengths['num_of_venues'], embedding_dim, padding_idx = 0)
        self.embeddingG3 = nn.Embedding(lengths['num_of_G3'], embedding_dim, padding_idx = 0)
        self.embeddingG4 = nn.Embedding(lengths['num_of_G4'], embedding_dim, padding_idx = 0)
        self.embeddingG5 = nn.Embedding(lengths['num_of_G5'], embedding_dim, padding_idx = 0)
        self.embeddingG6 = nn.Embedding(lengths['num_of_G6'], embedding_dim, padding_idx = 0)

        self.lstm = nn.LSTM(embedding_dim, hidden_size_lstm, num_layers=num_layers_lstm, batch_first=True)

        self.fc_user_venue = nn.Linear(hidden_size_lstm + embedding_dim, lengths['num_of_venues'])
        self.fc_G3 = nn.Linear(hidden_size_lstm + embedding_dim, lengths['num_of_G3'])
        self.fc_G4 = nn.Linear(hidden_size_lstm + embedding_dim, lengths['num_of_G4'])
        self.fc_G5 = nn.Linear(hidden_size_lstm + embedding_dim, lengths['num_of_G5'])
        self.fc_G6 = nn.Linear(hidden_size_lstm + embedding_dim, lengths['num_of_G6'])

        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, v, g3, g4, g5, g6, user_id):
        # Input x: (batch, seqlen)
        venue_embeddings = self.embeddingVenue(v)
        G3_emb = self.embeddingG3(g3)
        G4_emb = self.embeddingG4(g4)
        G5_emb = self.embeddingG5(g5)
        G6_emb = self.embeddingG6(g6)
        # Embed = (batch, seqlen, embed)
        # Input user_id: (batch)
        user_embeddings = self.embeddingUser(user_id)
        # Embed user = (batch, embed)
        lstm_out, _ = self.lstm(venue_embeddings)
        # Concatenate user embeddings to LSTM output
        lstm_out_with_user = torch.cat((lstm_out, user_embeddings.unsqueeze(1).repeat(1, lstm_out.size(1), 1)), dim=-1)
        lstm_out_with_G3 = torch.cat((lstm_out, G3_emb), dim = -1)
        lstm_out_with_G4 = torch.cat((lstm_out, G4_emb), dim = -1)
        lstm_out_with_G5 = torch.cat((lstm_out, G5_emb), dim = -1)
        lstm_out_with_G6 = torch.cat((lstm_out, G6_emb), dim = -1)

        # Apply dropout after the linear layer
        out_user = self.dropout(self.fc_user_venue(lstm_out_with_user))
        out_g3 = self.dropout(self.fc_G3(lstm_out_with_G3))
        out_g4 = self.dropout(self.fc_G4(lstm_out_with_G4))
        out_g5 = self.dropout(self.fc_G5(lstm_out_with_G5))
        out_g6 = self.dropout(self.fc_G6(lstm_out_with_G6))

        return (out_user, out_g3, out_g4, out_g5, out_g6)

    def compute_loss(self, outputs, v, g3, g4, g5, g6):
        loss_fn = nn.CrossEntropyLoss(reduction='mean')
        loss_POI = loss_fn(outputs[0].transpose(1, 2), v)  # Transpose outputs for correct shape
        loss_g3 = loss_fn(outputs[1].transpose(1, 2), g3)  # Transpose outputs for correct shape
        loss_g4 = loss_fn(outputs[2].transpose(1, 2), g4)  # Transpose outputs for correct shape
        loss_g5 = loss_fn(outputs[3].transpose(1, 2), g5)  # Transpose outputs for correct shape
        loss_g6 = loss_fn(outputs[4].transpose(1, 2), g6)  # Transpose outputs for correct shape
        loss_multitask = (loss_POI + loss_g3 + loss_g4 + loss_g5 + loss_g6) / 5
        return loss_multitask

    def training_step(self, batch, batch_idx):
        # Access the input sequences, target sequences, and user ID from the batch dictionary
        # Unpack the batch
        inputs_batch, targets_batch, user_id = batch

        # Unpack the sequences for each type
        venues, g3s, g4s, g5s, g6s = inputs_batch
        venue_tars, g3_tars, g4_tars, g5_tars, g6_tars = targets_batch
        # Assuming your model takes inputs and user embeddings

        outputs = self(venues, g3s, g4s, g5s, g6s, user_id)
        # Compute the loss based on the outputs and targets
        loss = self.compute_loss(outputs, venue_tars, g3_tars, g4_tars, g5_tars, g6_tars )

        # Log the training loss
        self.log('train_loss', loss)

        return loss

    def validation_step(self, batch, batch_idx):
        # Access the input sequences, target sequences, and user ID from the batch dictionary
        # Unpack the batch
        inputs_batch, targets_batch, user_id = batch
        # Unpack the sequences for each type
        venues, g3s, g4s, g5s, g6s = inputs_batch
        venue_tars, g3_tars, g4_tars, g5_tars, g6_tars = targets_batch
        # Assuming your model takes inputs and user embeddings
        outputs = self(venues, g3s, g4s, g5s, g6s, user_id)
        # Compute the loss based on the outputs and targets
        loss = self.compute_loss(outputs, venue_tars, g3_tars, g4_tars, g5_tars, g6_tars )
        self.log('val_loss', loss)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [21]:
# Instantiate the model
lengths = {'num_of_venues': len(df['Venue_ID_int'].unique()) + 1, # add one for padding token-index 0, recall each venue/g ID starts from 1
           'num_of_G3': len(df['g3_ID'].unique()) + 1,
           'num_of_G4': len(df['g4_ID'].unique()) + 1,
           'num_of_G5': len(df['g5_ID'].unique()) + 1,
           'num_of_G6': len(df['g6_ID'].unique()) + 1,
           'num_of_users': len(df['User_ID'].unique()), # No padding for users since they are not used as sequences but as single-value embedding in the concatenation
           }

hidden_size = 128
embed_dim = 128
num_layers = 2  # Number of LSTM layers

# Separate the sequences into training and validation
train_sequences, val_sequences = preprocess_dataset(df)

model = StackedLSTMClassifier(lengths, embed_dim, hidden_size, num_layers)

# Create separate datasets and dataloaders for training and validation
train_dataset = CustomDataset(train_sequences)
val_dataset = CustomDataset(val_sequences)

# Use custom collate function for handling variable-length sequences
batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate, num_workers=2)

In [22]:
from pytorch_lightning.loggers import TensorBoardLogger

# Instantiate TensorBoardLogger
tb_logger = TensorBoardLogger("logs", name="lstm_logs")

# Instantiate the Trainer with TensorBoardLogger
trainer = pl.Trainer(max_epochs=20, accelerator="gpu", devices=1, logger=tb_logger)

# Train the model
trainer.fit(model, train_dataloader, val_dataloaders=val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
   | Name           | Type      | Params
----------------------------------------------
0  | embeddingUser  | Embedding | 138 K 
1  | embeddingVenue | Embedding | 4.9 M 
2  | embeddingG3    | Embedding | 384   
3  | embeddingG4    | Embedding | 1.4 K 
4  | embeddingG5    | Embedding | 16.5 K
5  | embeddingG6    | Embedding | 317 K 
6  | lstm           | LSTM      | 264 K 
7  | fc_user_venue  | Linear    | 9.9 M 
8  | fc_G3          | Linear    | 771   
9  | fc_G4          | Linear    | 2.8 K 
10 | fc_G5          | Linear    | 33.2 K
11 | f

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

In [30]:
# Set the model to evaluation mode
import numpy as np
model.eval()

correct_task = np.array([0,0,0,0,0])
total_task = np.array([0,0,0,0,0])

with torch.no_grad():
    for batch in val_dataloader:
        inputs_batch, targets_batch, user_id = batch
        # Unpack the sequences for each type
        venues, g3s, g4s, g5s, g6s = inputs_batch
        venue_tars, g3_tars, g4_tars, g5_tars, g6_tars = targets_batch
        t = [venue_tars, g3_tars, g4_tars, g5_tars, g6_tars]
        # Assuming your model takes inputs and user embeddings
        outputs = model(venues, g3s, g4s, g5s, g6s, user_id)
        for i in range(5):
          total = 0
          correct = 0
          _, predicted = torch.max(outputs[i], 2)
          # Flatten predictions and targets to compute accuracy
          predicted = predicted.view(-1)  # Shape: (batch_size * sequence_length)
          targets = t[i].view(-1)      # Shape: (batch_size * sequence_length)
          correct += ((predicted == targets) & (targets != 0)).sum().item()
          total = (targets != 0).sum().item()
          correct_task[i] += correct
          total_task[i] += total

print(total_task, correct_task)
accuracy = correct_task / total_task

[44853 44853 44853 44853 44853] [ 4971 38793 35503 20604 10618]


In [31]:
print('Accuracy on validation set', (100 * accuracy))

Accuracy on validation set [11.08287071 86.48919805 79.1541257  45.93672664 23.67288699]
